## Installation

In [1]:
!python -m pip install datasets transformers tensorboard tensorflow ipywidgets opencv-python

In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [1]:
import tensorflow as tf
print(tf.config.experimental.list_physical_devices('GPU'))

model_id = "google/vit-base-patch16-224-in21k"

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import os
import datasets

def create_image_folder_dataset(root_path):
  """creates `Dataset` from image folder structure"""
  
  # get class names by folders names
  _CLASS_NAMES= os.listdir(root_path)
  # defines `datasets` features`
  features=datasets.Features({
                      "img": datasets.Image(),
                      "label": datasets.features.ClassLabel(names=_CLASS_NAMES),
                  })
  # temp list holding datapoints for creation
  img_data_files=[]
  label_data_files=[]
  # load images into list for creation
  for img_class in os.listdir(root_path):
    for img in os.listdir(os.path.join(root_path,img_class)):
      path_=os.path.join(root_path,img_class,img)
      img_data_files.append(path_)
      label_data_files.append(img_class)
  # create dataset
  ds = datasets.Dataset.from_dict({"img":img_data_files,"label":label_data_files},features=features)
  return ds


In [3]:
raw_dataset = create_image_folder_dataset("images_cropped")

In [4]:
img_class_labels = raw_dataset.features["label"].names
img_class_labels

['Svendborg Kommune',
 'Faaborg-Midtfyn Kommune',
 'Københavns Kommune1101',
 'Københavns Kommune0011',
 'Thisted Kommune',
 'Tårnby Kommune',
 'Morsø Kommune',
 'Frederikssund Kommune',
 'Horsens Kommune',
 'Hedensted Kommune',
 'Høje-Taastrup Kommune',
 'Lyngby-Taarbæk Kommune',
 'Københavns Kommune0010',
 'Københavns Kommune1100',
 'Dragør Kommune',
 'Næstved Kommune',
 'Solrød Kommune',
 'Helsingør Kommune',
 'Københavns Kommune100',
 'Aarhus Kommune10001',
 'Kerteminde Kommune',
 'Københavns Kommune1010',
 'Slagelse Kommune',
 'Fredensborg Kommune',
 'Københavns Kommune010',
 'Brøndby Kommune',
 'Assens Kommune',
 'Favrskov Kommune',
 'Københavns Kommune1011',
 'Lemvig Kommune',
 'Aarhus Kommune10000',
 'Samsø Kommune',
 'Aalborg Kommune0011',
 'Københavns Kommune011',
 'Langeland Kommune',
 'Fanø Kommune',
 'Aarhus Kommune01',
 'Rødovre Kommune',
 'Tønder Kommune',
 'Roskilde Kommune',
 'Ringkøbing-Skjern Kommune',
 'Vejle Kommune0',
 'Holstebro Kommune',
 'Aalborg Kommune01',
 '

In [5]:
from transformers import ViTFeatureExtractor
from tensorflow import keras 
from tensorflow.keras import layers


feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)

# learn more about data augmentation here: https://www.tensorflow.org/tutorials/images/data_augmentation
data_augmentation = keras.Sequential(
    [
        layers.Resizing(feature_extractor.size, feature_extractor.size),
        layers.Rescaling(1./255)
    ],
    name="data_augmentation",
)
# use keras image data augementation processing
def augmentation(examples):
    # print(examples["img"])
    examples["pixel_values"] = [data_augmentation(image) for image in examples["img"]]
    return examples


# basic processing (only resizing)
def process(examples):
    examples.update(feature_extractor(examples['img'], ))
    return examples

# we are also renaming our label col to labels to use `.to_tf_dataset` later
raw_dataset = raw_dataset.rename_column("label", "labels")



/opt/homebrew/anaconda3/envs/tf/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
2023-11-02 23:48:46.801615: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-11-02 23:48:46.801654: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-02 23:48:46.801662: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-02 23:48:46.801900: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-02 23:48:46.801931: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with

In [6]:
processed_dataset = raw_dataset.map(process, batched=True)
processed_dataset

Map:   0%|          | 0/19738 [00:00<?, ? examples/s]

Dataset({
    features: ['img', 'labels', 'pixel_values'],
    num_rows: 19738
})

In [7]:
# test size will be 15% of train dataset
test_size=.15

processed_dataset = processed_dataset.shuffle().train_test_split(test_size=test_size)
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['img', 'labels', 'pixel_values'],
        num_rows: 16777
    })
    test: Dataset({
        features: ['img', 'labels', 'pixel_values'],
        num_rows: 2961
    })
})

In [8]:
from huggingface_hub import HfFolder
import tensorflow as tf

id2label = {str(i): label for i, label in enumerate(img_class_labels)}
label2id = {v: k for k, v in id2label.items()}

num_train_epochs = 10
train_batch_size = 32
eval_batch_size = 32
learning_rate = 1e-6
weight_decay_rate=0.01
num_warmup_steps=0
output_dir=model_id.split("/")[1]
hub_token = HfFolder.get_token()
hub_model_id = f'dl-au-tamas-jedrek/{model_id.split("/")[1]}-street-view'


In [9]:
from transformers import DefaultDataCollator

# Data collator that will dynamically pad the inputs received, as well as the labels.
data_collator = DefaultDataCollator(return_tensors="tf")

# converting our train dataset to tf.data.Dataset
tf_train_dataset = processed_dataset["train"].to_tf_dataset(
   columns=['pixel_values'],
   label_cols=["labels"],
   shuffle=True,
   batch_size=train_batch_size,
   collate_fn=data_collator)

# converting our test dataset to tf.data.Dataset
tf_eval_dataset = processed_dataset["test"].to_tf_dataset(
   columns=['pixel_values'],
   label_cols=["labels"],
   shuffle=True,
   batch_size=eval_batch_size,
   collate_fn=data_collator)

/opt/homebrew/anaconda3/envs/tf/lib/python3.9/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [10]:
from transformers import TFViTForImageClassification, create_optimizer
import tensorflow as tf

# create optimizer wight weigh decay
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps,
)

# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)

# define loss
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# define metrics 
metrics=[
    tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(3, name="top-3-accuracy"),
]

# compile model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import os
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard as TensorboardCallback, EarlyStopping

callbacks = []
callbacks.append(TensorboardCallback(log_dir=os.path.join(output_dir, "logs")))
callbacks.append(EarlyStopping(monitor="val_accuracy",patience=1))
callbacks.append(PushToHubCallback(
    output_dir,
    hub_model_id=hub_model_id,
    hub_token=hub_token,
))



/Users/au724747/projects/dl-geolocation-vit/vit-base-patch16-224-in21k is already a clone of https://huggingface.co/dl-au-tamas-jedrek/vit-base-patch16-224-in21k-street-view. Make sure you pull the latest changes with `repo.git_pull()`.


In [12]:
from transformers import logging as transformers_logging
transformers_logging.set_verbosity_info()
train_results = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=callbacks,
    epochs=num_train_epochs,
    verbose=1
)

Epoch 1/10


2023-11-02 23:53:40.357459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


525/525 [==============================] - ETA: 0s - loss: 4.8673 - accuracy: 0.0084 - top-3-accuracy: 0.0255

Configuration saved in vit-base-patch16-224-in21k/config.json
Model weights saved in vit-base-patch16-224-in21k/tf_model.h5


525/525 [==============================] - 1600s 3s/step - loss: 4.8673 - accuracy: 0.0084 - top-3-accuracy: 0.0255 - val_loss: 4.8641 - val_accuracy: 0.0125 - val_top-3-accuracy: 0.0267
Epoch 2/10
525/525 [==============================] - ETA: 0s - loss: 4.8565 - accuracy: 0.0135 - top-3-accuracy: 0.0364

...

525/525 [==============================] - 1822s 3s/step - loss: 4.8143 - accuracy: 0.0321 - top-3-accuracy: 0.0767 - val_loss: 4.8290 - val_accuracy: 0.0277 - val_top-3-accuracy: 0.0594
Epoch 7/10
525/525 [==============================] - ETA: 0s - loss: 4.8079 - accuracy: 0.0355 - top-3-accuracy: 0.0820

Configuration saved in vit-base-patch16-224-in21k/config.json
Model weights saved in vit-base-patch16-224-in21k/tf_model.h5


525/525 [==============================] - 1607s 3s/step - loss: 4.8079 - accuracy: 0.0355 - top-3-accuracy: 0.0820 - val_loss: 4.8253 - val_accuracy: 0.0294 - val_top-3-accuracy: 0.0601
Epoch 8/10
525/525 [==============================] - ETA: 0s - loss: 4.8032 - accuracy: 0.0377 - top-3-accuracy: 0.0846

Configuration saved in vit-base-patch16-224-in21k/config.json
Model weights saved in vit-base-patch16-224-in21k/tf_model.h5


525/525 [==============================] - 1601s 3s/step - loss: 4.8032 - accuracy: 0.0377 - top-3-accuracy: 0.0846 - val_loss: 4.8227 - val_accuracy: 0.0297 - val_top-3-accuracy: 0.0621
Epoch 9/10
227/525 [===========>..................] - ETA: 4:18:18 - loss: 4.7992 - accuracy: 0.0380 - top-3-accuracy: 0.0895